# <div align='center'> 测试新架构API </div>

In [99]:
import requests
import json

In [100]:
hostname = 'gamma'
host = '10.255.20.227'
port = 8129

user = 'test'
uuid = '123456'

## 机器学习框架

### 一. 训练`/k12ai/framework/train`

#### A. 数据结构

1. 输入

```json
{
    "op":"", // [M] api具有子集操作, 如: "train.start", "train.stop"
    "user": "", // [M] 用户ID, api调用者, 如: 用户账号user_id(不含有特殊字符)
    "service_name": "", // [M] 服务名, k12ai由多个子框架组成, 需要指定具体服务, 如: "k12nlp", "k12cv"
    "service_uuid": "", // [M] 服务唯一标识码, 可以是用户项目的工程ID, project_id.
    "service_params": { // [M] 服务参数
        "dataset_reader": { // [M] 训练数据集处理配置
        },
        "validation_dataset_reader":{ // [O] 检验模型数据集的处理配置
        },
        "train_data_path": "", // [M] 训练数据集的路径
        "validation_data_path": "", // [O] 检验模型数据集的路径
        "model":{ // [M] 模型的配置 
        },
        "iterator":{ // [M] 训练/检验模型时采用的迭代方式
        },
        "trainer":{ // [M] 训练相关的参数配置
        }
    }
}
```

#### B. 配置实例

In [154]:
config = '''{
    "dataset_reader":{
        "type": "sst_tokens",
        "use_subtrees": true,
        "granularity": "5-class"
    },
    "validation_dataset_reader":{
        "type": "sst_tokens",
        "use_subtrees": false,
        "granularity": "5-class"
    },
    "train_data_path": "/data/datasets/nlp/sst/train.txt",
    "validation_data_path": "/data/datasets/nlp/sst/dev.txt",
    "test_data_path": "/data/datasets/nlp/sst/test.txt",
    "model": {
        "type": "bcn",
        "text_field_embedder": {
            "token_embedders": {
                "tokens": {
                    "type": "embedding",
                    "embedding_dim": 50,
                    "pretrained_file": "/data/datasets/nlp/glove/glove.6B.50d.txt.gz",
                    "trainable": false
                }
            }
        },
        "embedding_dropout": 0.25,
        "pre_encode_feedforward": {
            "input_dim": 50,
            "num_layers": 1,
            "hidden_dims": [50],
            "activations": ["relu"],
            "dropout": [0.25]
        },
        "encoder": {
            "type": "lstm",
            "input_size": 50,
            "hidden_size": 50,
            "num_layers": 1,
            "bidirectional": true
        },
        "integrator": {
            "type": "lstm",
            "input_size": 300,
            "hidden_size": 50,
            "num_layers": 1,
            "bidirectional": true
        },
        "integrator_dropout": 0.1,
        "output_layer": {
            "input_dim": 400,
            "num_layers": 3,
            "output_dims": [200, 100, 5],
            "pool_sizes": 4,
            "dropout": [0.2, 0.3, 0.0]
        }
    },
    "iterator": {
        "type": "bucket",
        "sorting_keys": [["tokens", "num_tokens"]],
        "batch_size" : 2
    },
    "trainer": {
        "num_epochs": 10,
        "patience": 2,
        "grad_norm": 5.0,
        "validation_metric": "+accuracy",
        "optimizer": {
            "type": "adam",
            "lr": 0.001
        },
        "cuda_device": 0
    }
}'''
# json.loads(config)

#### C. 启动训练

In [155]:
data = '''{
    "op":"train.start",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s",
    "service_params": %s
}''' % (user, uuid, config)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
start_res = requests.post(url=api, json=json.loads(data))
start_res.text

'{"content": {"result": {"op": "train.start", "exec": "success", "cache_dir": "/data/users/test/123456"}}, "code": 100000}'

[操作是异步的, 训练过程结果到Consul数据中心查看](http://gamma:8500/ui/gamma/kv/test/123456/)

#### D. 停止训练

In [96]:
data = '''{
    "op":"train.stop",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s",
    "service_params": {}
}''' % (user, uuid)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
stop_res = requests.post(url=api, json=json.loads(data))
stop_res.text

'{"content": {"result": 0}, "code": 100000}'

[操作是异步的, 真实启动的结果到Consul数据中心查看](http://gamma:8500/ui/gamma/kv/test/123456/)

### 二. 评估`/k12ai/framework/evaluate`

#### A. 数据结构

1. 输入

```json
{
    "op":"", // [M] api具有子集操作, 如: "evaluate.start", "evaluate.stop"
    "user": "", // [M] 用户ID, api调用者, 如: 用户账号user_id(不含有特殊字符)
    "service_name": "", // [M] 服务名, k12ai由多个子框架组成, 需要指定具体服务, 如: "k12nlp", "k12cv"
    "service_uuid": "", // [M] 服务唯一标识码, 可以是用户项目的工程ID, project_id.
    "service_params": { // [M] 服务参数
        "input_file": "" // [M] 评估/测试数据集的路径
        "output_file": "" // [O] 指定评估最终结果的metrics存储文件
    }
}
```

2. 输出

```json
{
    "code": 100000, // [M] 操作结果码, (异步)TODO: 100000只能说明接口调用是通的, 不能说明训练是成功的.
    "content": {  // [O] 操作结果详情
        "result": {
            "op": "evaluate.stop",
            "exec": "success"
        }
    }
}
```

#### B. 启动评估

In [121]:
data = '''{
    "op":"evaluate.start",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s",
    "service_params": {
        "input_file": "/data/datasets/nlp/sst/test.txt"
    }
}''' % (user, uuid)

api = 'http://%s:%d/k12ai/framework/evaluate' % (host, port)
requests.post(url=api, json=json.loads(data)).text

'{"content": {"result": {"op": "evaluate.start", "exec": "success"}}, "code": 100000}'

[操作是异步的, 真实启动的结果到Consul数据中心查看](http://gamma:8500/ui/gamma/kv/test/123456/)

评估过程中消息如下:

```json
{
    "version": "0.1.0", // 消息体的版本号, 为了兼容设定
    "op": "evaluate.start", // 操作
    "user": "test", // 用户ID
    "service_uuid": "123456", // 工程ID
    "timestamp": 1574224024170, // 时间戳
    "datetime": "2019-11-20 12:27:04", // 日期
    "message": { // 具体消息, 阶段不同, 消息结构和内容都有可能发生变化
        "accuracy": 0.4027777777777778,
        "accuracy3": 0.8255555555555556,
        "loss": 1.3810093342098926
    }
}
```

#### C. 停止评估

In [122]:
data = '''{
    "op":"evaluate.stop",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s"
}''' % (user, uuid)

api = 'http://%s:%d/k12ai/framework/evaluate' % (host, port)
requests.post(url=api, json=json.loads(data)).text

'{"content": {"result": {"op": "evaluate.stop", "exec": "success"}}, "code": 100000}'

[操作是异步的, 真实启动的结果到Consul数据中心查看](http://gamma:8500/ui/gamma/kv/test/123456/)

停止评估任务, 如果该任务已经结束, 返回结果如下:

```json
{
    "version": "0.1.0",
    "op": "evaluate.stop",
    "user": "test",
    "service_uuid": "123456",
    "timestamp": 1574224373067,
    "datetime": "2019-11-20 12:32:53"
        "result": {
            "code": -1,
            "err": "404 Client Error: Not Found (\"No such container: evaluate-test-123456\")"
        }
    }
}
```

### 3. 预测`/k12ai/framework/predict`

#### A. 数据结构

1. 输入

```json
{
    "op":"", // [M] api具有子集操作, 如: "predict.start", "predict.stop"
    "user": "", // [M] 用户ID, api调用者, 如: 用户账号user_id(不含有特殊字符)
    "service_name": "", // [M] 服务名, k12ai由多个子框架组成, 需要指定具体服务, 如: "k12nlp", "k12cv"
    "service_uuid": "", // [M] 服务唯一标识码, 可以是用户项目的工程ID, project_id.
    "service_params": { // [M] 服务参数
        "input_type": "", // [M] 指定预测文本的类型, 1. "text": 文本串; 2. "file": 文件
        "input_file": "", // [C] 当input_type=file; 指定预测文本路径
        "input_text": "", // [C] 当input_type=text; 指定预测文本内容
        "output_file": "", // [O] 指定最终结果输出的存储文件
        "batch_size": 32, // [0] 指定预测时使用的batch大小
        "predictor": "", // [O] 指定预测方法类型, 如果不指定则从训练配置config.json中读取
    }
}
```

#### B. 启动预测

In [148]:
text = '{"sentence": "a very well-made, funny and entertaining picture."}'
data = '''{
    "op":"predict.start",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s",
    "service_params": {
        "input_type": "text",
        "input_text": %s,
        "predictor": "text_classifier"
    }
}''' % (user, uuid, json.dumps(text))

api = 'http://%s:%d/k12ai/framework/predict' % (host, port)
requests.post(url=api, json=json.loads(data)).text

'{"content": {"result": {"op": "predict.start", "exec": "success", "cache_dir": "/data/users/test/123456"}}, "code": 100000}'

[操作是异步的, 真实启动的结果到Consul数据中心查看](http://gamma:8500/ui/gamma/kv/test/123456/)

输出样例:

```json
{
    "version": "0.1.0",
    "op": "predict.start",
    "user": "test",
    "service_uuid": "123456",
    "timestamp": 1574240762485,
    "datetime": "2019-11-20 17:06:02",
    "message": {
        "index": 0,  // 表示本次预测是第几个实例
        "prediction": "{\"logits\": [-1.4716334342956543, 1.646652340888977, -2.384199857711792, 2.6578245162963867, -4.671465873718262], \"class_probabilities\": [0.011601723730564117, 0.26228705048561096, 0.004658004269003868, 0.7209802269935608, 0.00047299108700826764], \"label\": \"4\"}\n"
    }
}
```

#### C. 停止预测

In [153]:
data = '''{
    "op":"predict.stop",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s"
}''' % (user, uuid)

api = 'http://%s:%d/k12ai/framework/predict' % (host, port)
requests.post(url=api, json=json.loads(data)).text

'{"content": {"result": {"op": "predict.stop", "exec": "success"}}, "code": 100000}'

[操作是异步的, 真实启动的结果到Consul数据中心查看](http://gamma:8500/ui/gamma/kv/test/123456/)

输出样例:

```json
{
    "version": "0.1.0",
    "op": "predict.stop",
    "user": "test",
    "service_uuid": "123456",
    "timestamp": 1574241361253,
    "datetime": "2019-11-20 17:16:01",
    "message": {
        "result": {
            "code": 0,
            "id": "867680f57c" // Container ID
        }
    }
}
```

## 平台资源管理